# Final Project

## 파이썬 환경 설정 및 확인

In [ ]:
# C:/Users/TECH2_22/이도원/usingcloud_final/venv/Scripts/python.exe

In [6]:
import os

os.getcwd()

'c:\\Users\\4ser\\Documents\\github\\usingcloud_final'

In [7]:
print('hello world')

hello world


In [8]:
!python --version

Python 3.8.8


In [ ]:
# !pip install datasets
# !pip install transformers
# !pip install rouge_score

### CUDA 설정

In [28]:
import torch

torch.cuda.is_available()

False

## 데이터 로드 및 설정

In [10]:
import pandas as pd
import numpy as np
import json

with open('./data/train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open('./data/valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])
    

In [11]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

In [12]:
import pyarrow as pa
import pyarrow.dataset as ds

from datasets import Dataset

train_df['body'] = train_df.text.apply(lambda x : sentence_validation(extract_body(x)))
valid_df['body'] = valid_df.text.apply(lambda x : sentence_validation(extract_body(x)))

train_df.abstractive = train_df.abstractive.apply(lambda x : x[0])
valid_df.abstractive = valid_df.abstractive.apply(lambda x : x[0])

train_df.drop(train_df.body.str.len().sort_values(ascending=False).head(1).index[0], inplace=True)


df = train_df.copy()
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))
del df

train_df.head(2)

c:\Users\4ser\Documents\github\usingcloud_final\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive,body
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...


## eda

In [16]:
train_df.head(3)

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive,body
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,전남드래곤즈(사장 신승재)는 지난 4일 구봉산 해맞이 행사를 통해 새해 각오를 다졌...


In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243982 entries, 0 to 243982
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   id                       243982 non-null  object
 1   category                 243982 non-null  object
 2   media_type               243982 non-null  object
 3   media_sub_type           243982 non-null  object
 4   media_name               243982 non-null  object
 5   size                     243982 non-null  object
 6   char_count               243982 non-null  object
 7   publish_date             243982 non-null  object
 8   title                    243982 non-null  object
 9   text                     243982 non-null  object
 10  annotator_id             243982 non-null  int64 
 11  document_quality_scores  243982 non-null  object
 12  extractive               243982 non-null  object
 13  abstractive              243982 non-null  object
 14  body                

In [18]:
train_df.shape

(243982, 15)

In [19]:
train_df.head(1).T

,0
id,290741778
category,종합
media_type,online
media_sub_type,지역지
media_name,광양신문
size,small
char_count,927
publish_date,2018-01-05 18:54:55
title,"논 타작물 재배, 2월 말까지 신청하세요"
text,"[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ..."


In [20]:
train_df.isnull().sum()

id                         0
category                   0
media_type                 0
media_sub_type             0
media_name                 0
size                       0
char_count                 0
publish_date               0
title                      0
text                       0
annotator_id               0
document_quality_scores    0
extractive                 0
abstractive                0
body                       0
dtype: int64

In [21]:
train_df.columns

Index(['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size',
       'char_count', 'publish_date', 'title', 'text', 'annotator_id',
       'document_quality_scores', 'extractive', 'abstractive', 'body'],
      dtype='object')

In [ ]:
train_df.body.str.len().sort_values(ascending=False).head(3)

36893     1963
182051    1959
187153    1955
Name: body, dtype: int64

In [ ]:
train_df.abstractive.str.len().sort_values(ascending=False).head(3)

149185    716
152576    638
170525    610
Name: abstractive, dtype: int64

## 전처리

In [13]:
from keybert import KeyBERT
from kiwipiepy import Kiwi


# 문장 추출
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence


# 문장 유효성 검사
def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)


# 명사 추출
def noun_extractor(text):
    kiwi = Kiwi()
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results


# 이건 뭐지
def BERT(title):
    bow = []
    array_text = pd.DataFrame(df[df['title'] == title]['text']).to_numpy()
    
    kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
    for j in range(len(array_text)):
        keywords = kw_extractor.extract_keywords(array_text[j][0])
        bow.append(keywords)
    
    new_bow = []
    for i in range(0, len(bow)):
        for j in range(len(bow[i])):
            new_bow.append(bow[i][j])
            
    keyword = pd.DataFrame(new_bow, columns=['keyword', 'weight'])
    print(keyword.groupby('keyword').agg('sum').sort_values('weight', ascending=False).head(20))

In [16]:
from keybert import KeyBERT
from transformers import BertModel

text="""
김영하가 『살인자의 기억법』 이후 9 년 만에 내놓는 장편소설 『작별인사』는 그리 멀지 않은 미래를 배경으로, 별안간 삶이 송두리째 뒤흔들린 한 소년의 여정을 좇는다. 
유명한 IT 기업의 연구원인 아버지와 쾌적하고 평화롭게 살아가던 철이는 어느날 갑자기 수용소로 끌려가 난생처음 날것의 감정으로 가득한 혼돈의 세계에 맞닥뜨리게 되면서 정신적, 신체적 위기에 직면한다. 
동시에 자신처럼 사회에서 배제된 자들을 만나 처음으로 생생한 소속감을 느끼고 따뜻한 우정도 싹틔운다. 철이는 그들과 함께 수용소를 탈출하여 집으로 돌아가기 위해 길을 떠나지만 그 여정에는 피할 수 없는 질문이 기다리고 있다.
세계보건기구 WHO가 팬데믹을 선언한 지 2년이 지나서야 작가는 『작별인사』의 개작을 마쳤다. 420매 분량이던 원고는 약 800매로 늘었고, 주제도 완전히 달라졌다. 
‘인간을 인간으로 만드는 것은 무엇인가?’, ‘인간과 인간이 아닌 존재들을 가르는 경계는 어디인가’를 묻던 소설은 ‘삶이란 과연 계속될 가치가 있는 것인가?’, 
‘세상에 만연한 고통을 어떻게 하면 줄일 수 있을 것인가’, ‘어쩔 수 없이 태어났다면 어떻게 살고 어떻게 죽어야 할 것인가’와 같은 질문을 던지는 이야기로 바뀌었다.
팬데믹이 개작에 영향을 주었을 수도 있고, 원래 『작별인사』의 구상에 담긴 어떤 맹아가 오랜 개작을 거치며 발아했는지도 모른다. 그것에 대해 작가는 이렇게 말하고 있다.
마치 제목이 어떤 마력이 있어서 나로 하여금 자기에게 어울리는 이야기로 다시 쓰도록 한 것 같은 느낌이다. 탈고를 하고 얼마 지나지 않아 원고를 다시 읽어보았다. 
이제야 비로소 애초에 내가 쓰려고 했던 어떤 것이 제대로, 남김 없이 다 흘러나왔다는 생각이 들었다. _’작가의 말’에서
전면적인 수정을 통해 2022년의 『작별인사』는 2020년의 『작별인사』를 마치 시놉시스나 초고처럼 보이게 할 정도로 확연하게 달라졌다. 그리고 김영하의 이전 문학 세계와의 연결점들이 분명해졌다.
"""

model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('장편소설', 0.531),
 ('소설은', 0.5247),
 ('위기에', 0.5216),
 ('삶이란', 0.5162),
 ('삶이', 0.5157)]

In [ ]:
# kiwi = Kiwi()
# kiwi.analyze(text)

# noun_extractor(text)

### 문서 요약

In [14]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

c:\Users\4ser\Documents\github\usingcloud_final\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\4ser\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of la

### 필요 함수 선언

In [15]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["body"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # 데이터가 손실될 경우에 앞의 단어가 아니라 뒤의 단어가 삭제되도록 하고싶다면 truncating이라는 인자를 사용합니다. truncating='post'를 사용할 경우 뒤의 단어가 삭제됩니다.
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstractive"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
hg_dataset

Dataset({
    features: ['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size', 'char_count', 'publish_date', 'title', 'text', 'annotator_id', 'document_quality_scores', 'extractive', 'abstractive', 'body', '__index_level_0__'],
    num_rows: 243982
})

In [16]:
prefix = ""
max_input_length = 2000
max_target_length = 750

# preprocess_function(hg_dataset)

In [17]:
tokenized_data = hg_dataset.map(preprocess_function, batched=True)
tokenized_data

Map:   0%|          | 0/243982 [00:00<?, ? examples/s]c:\Users\4ser\Documents\github\usingcloud_final\venv\lib\site-packages\transformers\tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Dataset({
    features: ['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size', 'char_count', 'publish_date', 'title', 'text', 'annotator_id', 'document_quality_scores', 'extractive', 'abstractive', 'body', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 243982
})

### 모델링

In [18]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_metric

metric = load_metric("rouge")

# model = AutoModelForSeq2SeqLM.from_pretrained('gogamza/kobart-base-v2')

C:\Users\4ser\AppData\Local\Temp\ipykernel_25428\4195763225.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [20]:
from transformers import Seq2SeqTrainingArguments

batch_size = 4
model_name = 'awesome_model'
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    # fp16=True,                      # CUDA 설정을 완료한 후 사용가능.
    push_to_hub=False,
)


In [21]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
import nltk
import numpy as np
nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4ser\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
tokenized_data

Dataset({
    features: ['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size', 'char_count', 'publish_date', 'title', 'text', 'annotator_id', 'document_quality_scores', 'extractive', 'abstractive', 'body', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 243982
})

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

## 여기가 끝

### 여기서부터 KoBART

In [19]:
import sys

local_vars = list(locals().items())

for var, obj in local_vars:
    obj_size = sys.getsizeof(obj) # Return the size of object in bytes -> 1064 * 1064 bytes == 1 MB
    if (obj_size > 1064 * 1064) and (not var.startswith('_')): 
        print(var, int(obj_size / (1064 * 1064)), 'MB')

train_df 1348 MB
valid_df 98 MB
